<a href="https://colab.research.google.com/github/richmondvan/melanoma-detection/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

Import all modules and mount Google Drive

In [0]:
# Must be run every time!
import pathlib
import math
import os
import pickle
from google.colab import drive 
import tensorflow_hub as hub

import tensorflow as tf #nightly
from tensorflow.keras import models, layers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

drive.mount('/content/gdrive') 

Copy dataset to local drive

In [0]:
%cp -R /content/gdrive/My Drive/Dataset/DatasetSorted /Dataset

Prepare datasets

In [0]:
PATH = "/Dataset/DatasetSorted/"

TRAINING_PATH = pathlib.Path(PATH + "training/")
VALIDATION_PATH = pathlib.Path(PATH + "validation/")
TEST_PATH = pathlib.Path(PATH + "test/")



train_image_generator = ImageDataGenerator(
    rescale=1./255,
    brightness_range=(0.95, 1.05),
    horizontal_flip=True,
    vertical_flip=True) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True) # Generator for our validation data

batch_size = 32
epochs = 15
IMG_HEIGHT = 600
IMG_WIDTH = 600

TRAIN_LEN = len(list(TRAINING_PATH.glob("*/*.jpg")))
VALID_LEN = len(list(VALIDATION_PATH.glob("*/*.jpg")))

print(TRAIN_LEN)
print(VALID_LEN)

CLASS_NAMES = ['benign', 'malignant']

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=TRAINING_PATH,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary',
                                                           classes=CLASS_NAMES)

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=VALIDATION_PATH,
                                                              shuffle=True,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary',
                                                              classes=CLASS_NAMES)

Prepare metrics and weights

In [0]:
numBenign = len(list(TRAINING_PATH.glob("benign/*.jpg")))
numMalignant = len(list(TRAINING_PATH.glob("malignant/*.jpg")))
total = numBenign + numMalignant
weight_for_0 = (1 / numBenign)*(total)/2.0 
weight_for_1 = (1.5 / numMalignant)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1


METRICS = [
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'), 
      metrics.BinaryAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
      metrics.AUC(name='auc'),
]

Prepare model

In [0]:
NEURONS_PER_LAYER = 1024

model = models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/efficientnet/b7/feature-vector/1", trainable=False),
    layers.Dropout(0.1),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(NEURONS_PER_LAYER, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=METRICS)

model.build([None, IMG_HEIGHT, IMG_WIDTH, 3])
model.summary()



Load model weights and last epoch

In [0]:
EPOCH_FILEPATH = f"/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_4_epochnum"
infile = open(EPOCH_FILEPATH, 'rb')
try: 
    epoch = pickle.load(infile)
    model.load_weights(f"/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_4_epoch{epoch}.h5")
except: 
    epoch = 0


infile.close()

Train model

In [0]:
for i in range(epoch, 150):
    outfile = open(EPOCH_FILEPATH, 'wb')
    history = model.fit(x=train_data_gen, 
                        epochs=i+1, 
                        initial_epoch=i, 
                        verbose=1, 
                        validation_data=val_data_gen, 
                        validation_steps=VALID_LEN // batch_size, 
                        steps_per_epoch=TRAIN_LEN // batch_size, 
                        class_weight=class_weight)
    model.save_weights(f"/content/gdrive/My Drive/Dataset/enet{NEURONS_PER_LAYER}_4/epoch{i + 1}.h5")
    pickle.dump(i+1, outfile)
    outfile.close()